In [1]:
import numpy as np 
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
train_path = 'dataset/train'
valid_path = 'dataset/valid'
test_path = 'dataset/test'

In [2]:
def preprocess(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img = tf.keras.preprocessing.image.img_to_array(img,dtype=np.uint8)
    #img = np.expand_dims(img, axis=0)
    img = np.array(img,dtype=np.uint8)
    img = cv2.GaussianBlur(img, (3, 3), 7)
    mean = int(np.mean(img))
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            img[i,j] = max(0,img[i,j]-mean)
    clahe = cv2.createCLAHE()
    img = clahe.apply(img)
    img = img.reshape(-1, 224, 224, 1)
    return img/255.0
#-1
train_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(train_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=True)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(valid_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=preprocess).flow_from_directory(test_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10,shuffle=False)

Found 2100 images belonging to 5 classes.
Found 696 images belonging to 5 classes.
Found 900 images belonging to 5 classes.


In [ ]:
import matplotlib.pyplot as plt
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
imgs, labels = next(train_batches)
plots(imgs, titles=labels)

In [ ]:
#train_batches = train_batches.reshape(-1, 224, 224, 1)
imgs, labels = next(train_batches)
print(imgs[0].shape)
#print(type(train_batches))

In [ ]:
######1-vgg16 preprocessing
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(train_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=4)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(test_path, target_size=(224,224), classes=['0', '1','2','3','4'], batch_size=10)

In [3]:
#0
from tensorflow.keras.layers import Dropout
from keras import regularizers
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.1)))
model.add(Dropout(0.5))
model.add(MaxPool2D(pool_size=(2,2),strides=2))
model.add(Flatten())
model.add(Dense(units=5, activation="softmax"))
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#1
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#2 3 3 2
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=5, activation="softmax"))
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#2-OG
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=5, activation="softmax"))
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
dropout (Dropout)            (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 256)     147712    
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 256)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 256)     590080    
_________________________________________________________________
dropout_2 (Dropout)          (None, 112, 112, 256)     0

In [5]:
model.fit(train_batches, batch_size=10 , validation_data=valid_batches, epochs=10, verbose=2) 

Epoch 1/10
210/210 - 1542s - loss: 47.8247 - accuracy: 0.2838 - val_loss: 19.9765 - val_accuracy: 0.3247
Epoch 2/10
210/210 - 1581s - loss: 10.1492 - accuracy: 0.3190 - val_loss: 4.7036 - val_accuracy: 0.3017
Epoch 3/10
210/210 - 1583s - loss: 2.9457 - accuracy: 0.3848 - val_loss: 2.2841 - val_accuracy: 0.2601
Epoch 4/10
210/210 - 1578s - loss: 1.8065 - accuracy: 0.4067 - val_loss: 1.8981 - val_accuracy: 0.1638
Epoch 5/10


In [ ]:
from tensorflow.keras.layers import Dropout
from keras import regularizers
model2 = Sequential()
model2.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu",kernel_regularizer=regularizers.l2(0.01)))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Flatten())
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dropout(0.5))
model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=5, activation="softmax"))
model2.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
model2.fit(train_batches, batch_size=10 , validation_data=valid_batches, epochs=10, verbose=2)

In [ ]:
'''
steps_per_epoch = int(number_of_train_samples / batch_size)
val_steps = int(number_of_val_samples / batch_size)
'''
#new
model.fit_generator(train_batches, steps_per_epoch=210, validation_data=valid_batches, validation_steps=69, epochs=10, verbose=2)

In [ ]:
model.save('dataset/OF255Clean.h5')

In [ ]:
predictions = model.predict(test_batches, steps=1, verbose=0)

In [ ]:
np.argmax(predictions,axis=-1)

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
plots(imgs, titles=labels)